## model setting

In [ ]:
pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_c

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

## setting

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#data path
path = '/content/drive/MyDrive/python/공공데이터 공모전/data/'
#load data
corp = pd.read_json(path+'mapping30.json', dtype = {'CODE':str})
hs_df = pd.read_json(path+'hs_transformer_embedding.json').explode('HS부호')

## preprocessing

In [ ]:
def change_key_to_float(di):
  new = {}
  for key in di.keys():
    new[float(key)] = di[key]
  return new

corp['matched'] = corp['matched'].apply(lambda x : change_key_to_float(x))
corp

,DSC,LLMimport,transformer_embedding,matched,maxscore,hs2017
0,"automotive repair shops, nec specialized auto...","Brake linings, carburetors, radiators, fuel fi...","[0.0058483728, -0.0131848697, -0.005108098, 0....","{0.6494507475436267: [7320201000], 0.583288167...",0.649451,0
1,"general farms, primarily animals, nsk derives...","Feeding equipment, veterinary supplies, and fa...","[0.0656781793, 0.0379473008, -0.01657006890000...","{0.6073210918483304: [407190000, 407290000, 40...",0.607321,0
2,fish and seafoods the wholesale distribution ...,"Cooler units, pallets, crates, boxes, gloves, ...","[0.0419675633, -0.0011303811, -0.0371737815, -...","{0.6609429034116933: [2104102000], 0.650530437...",0.660943,0
3,"new and used car dealers, nsk manufactures a ...","Steel alloys for car chassis and body panels, ...","[0.006544634200000001, -0.0115391007, -0.01442...","{0.6839825584977619: [7320201000], 0.597469665...",0.683983,0
4,"automotive stampings, nsk manufacturing autom...","Steel sheets, Aluminum alloys, Copper alloys, ...","[0.0191266164, -0.0810126439, -0.0019861253, 0...","{0.6344416678820545: [7320201000], 0.630219728...",0.634442,0
...,...,...,...,...,...,...
1713,"printing and writing paper, nsk the wholesale...","Ink, toners, and printing supplies.","[0.0713708326, -0.0066772108000000005, 0.01651...","{0.6356510586659162: [4821100000], 0.609555263...",0.635651,0
1714,leather and sheep-lined clothing manufacturin...,"Leather hides, Sheep pelts or wool, Thread (sp...","[-0.0333148204, -0.012678897000000001, 0.01240...","{0.7126465277629943: [6209201000, 6209301000, ...",0.712647,0
1715,"electronic parts and equipment, nec, nsk the ...","Integrated circuits, Microprocessors, Printed ...","[0.0356280208, -0.0699515864, -0.0032274586, -...","{0.6810985356731969: [8517719000], 0.664486383...",0.681099,0
1716,"bottled and canned soft drinks, nsk manufactu...","Glass or plastic bottles, Aluminum cans, Label...","[0.0669452697, -0.0210717097, -0.0233917888, -...","{0.6079746328721736: [3824996300], 0.580094565...",0.607975,0


In [ ]:
hs_dict = {}
for index, row in hs_df.iterrows():
    hs_dict[row["HS부호"]] = row["품목명"]

In [ ]:
corp['cross_result'] = [{} for _ in range(len(corp))]

## reranking

In [ ]:
for index, entry in corp.iterrows():
    dsc = entry["DSC"]
    matched = entry["matched"]

    pairs = []
    hs_codes_list = {}

    for hs_codes in matched.values():
        for hs_code in hs_codes:
            if hs_code in hs_dict:
                document = hs_dict[hs_code]
                pairs.append((dsc, document))
                if document not in hs_codes_list:
                    hs_codes_list[document] = []
                hs_codes_list[document].extend(hs_codes)
                break

    if pairs:

        scores = reranker.compute_score([(dsc, document) for dsc, document in pairs])

        cross_results = {}
        for (query, document), score in zip(pairs, scores):
            if score not in cross_results:
                cross_results[score] = []
            cross_results[score].append(hs_codes_list[document])

        sorted_cross_result = {k: cross_results[k] for k in sorted(cross_results, reverse=True)}
        corp.at[index, "cross_result"] = sorted_cross_result

In [ ]:
# 정렬, 리스트 풀기
for index, entry in corp.iterrows():
    cross_result = entry["cross_result"]

    for score, hs_codes_list in cross_result.items():
        cross_result[score] = [item for sublist in hs_codes_list for item in sublist]

    sorted_cross_result = {k: cross_result[k] for k in sorted(cross_result, reverse=True)}
    corp.at[index, "cross_result"] = sorted_cross_result

In [ ]:
corp.to_json('Reranker.json')